In [1]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import scipy.stats as st
from statsmodels.stats.multitest import multipletests
from arch.bootstrap import IIDBootstrap
import seaborn as sns
from sklearn import datasets
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 4)

Загрузим данные:

In [2]:
irises = datasets.load_iris()

In [3]:
data = irises.data
target = irises.target

Создадим подвыборки, соответсвующие одному классу:

In [5]:
setosa = data[target==0]
versicolor = data[target==1]
virginica = data[target==2]

Для кажого признака создадим массивы. Колонке массива соответствуют данные одного класса.

In [6]:
irises.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [7]:
s_l = np.vstack((setosa[:, 0], versicolor[:, 0], virginica[:, 0]))
s_w = np.vstack((setosa[:, 1], versicolor[:, 1], virginica[:, 1]))
p_l = np.vstack((setosa[:, 2], versicolor[:, 2], virginica[:, 2]))
p_w = np.vstack((setosa[:, 3], versicolor[:, 3], virginica[:, 3]))

Проверим, совпадают ли характеристики в среднем:
$$H^i_0: E(X^i_1)=E(X^i_2)$$

Будем использовать поправку Холма.

In [8]:
p_values= []
for feat in [s_l, s_w, p_l, p_w]:
    p_v_f = []
    for i in range(3):
        for j in range(i+1, 3):
            test_result = st.ttest_ind(feat[i], feat[j])
            p_v_f += [test_result[1]] 
    p_values.append(p_v_f)

In [9]:
for i in range(4):
    rejections = multipletests(p_values[i], method='holm')[0].sum()
    if rejections > 0:
        print('H0 rejected for feature', irises.feature_names[i])

H0 rejected for feature sepal length (cm)
H0 rejected for feature sepal width (cm)
H0 rejected for feature petal length (cm)
H0 rejected for feature petal width (cm)


Таким образом, характеристики в среднем не совпадают.

Построим доверительные интервалы. Предполагаем, что данные разных цветков независимые.

In [10]:
for k, feat in enumerate([s_l, s_w, p_l, p_w]):
    print('Feature -', irises.feature_names[k], ':')
    for i in range(3):
        for j in range(i+1, 3):
            bs = IIDBootstrap(feat[i]-feat[j])
            confint = bs.conf_int(np.mean, 1000, method='bca')
            print('Conf_int for mean', irises.target_names[i]+'-'+irises.target_names[j], ':', confint) 
    print('----------------------------------------------------')

Feature - sepal length (cm) :
Conf_int for mean setosa-versicolor : [[-1.12447931]
 [-0.75971516]]
Conf_int for mean setosa-virginica : [[-1.786     ]
 [-1.40197232]]
Conf_int for mean versicolor-virginica : [[-0.90523526]
 [-0.42448137]]
----------------------------------------------------
Feature - sepal width (cm) :
Conf_int for mean setosa-versicolor : [[0.50747943]
 [0.8       ]]
Conf_int for mean setosa-virginica : [[0.318]
 [0.588]]
Conf_int for mean versicolor-virginica : [[-0.33746366]
 [-0.078     ]]
----------------------------------------------------
Feature - petal length (cm) :
Conf_int for mean setosa-versicolor : [[-2.9306582 ]
 [-2.63956376]]
Conf_int for mean setosa-virginica : [[-4.24863306]
 [-3.94446376]]
Conf_int for mean versicolor-virginica : [[-1.49747791]
 [-1.09      ]]
----------------------------------------------------
Feature - petal width (cm) :
Conf_int for mean setosa-versicolor : [[-1.146]
 [-1.018]]
Conf_int for mean setosa-virginica : [[-1.85687742]

Как видно из полученных доверительных интервалов:
- среднее отличие по признаку sepal length варьируется от $0.5$ до $1.5$
- среднее отличие по признаку sepal width варьируется от $0.1$ до $0.7$
- среднее отличие по признаку petal length варьируется от $1.1$ до $4.2$
- среднее отличие по признаку sepal length варьируется от $0.65$ до $1.8$
